In [1]:
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
import torch
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from PIL import Image
import xml.etree.ElementTree as ET

import torchvision
import torchvision.transforms as T
from torchvision.models.detection import FasterRCNN, fasterrcnn_resnet50_fpn
from torchvision.models.detection.rpn import AnchorGenerator
from torch.utils.data import Dataset, DataLoader
from torch.optim import Adam


In [3]:
# Paths for annotations and images
annotations_path = '/content/drive/MyDrive/mini project/annotations'
images_path = '/content/drive/MyDrive/mini project/images'

In [4]:
class UATDDataset(Dataset):
    def __init__(self, image_dir, annotation_dir, transforms=None):
        self.image_dir = image_dir
        self.annotation_dir = annotation_dir
        self.transforms = transforms
        self.images = sorted([f for f in os.listdir(self.image_dir) if f.endswith('.bmp')])
        self.annotations = sorted([f for f in os.listdir(self.annotation_dir) if f.endswith('.xml')])

        assert len(self.images) == len(self.annotations), "Mismatch between images and annotations count!"

    def __getitem__(self, idx):
        # Paths to image and annotation
        img_path = os.path.join(self.image_dir, self.images[idx])
        ann_path = os.path.join(self.annotation_dir, self.annotations[idx])

        # Load image
        img = Image.open(img_path).convert("RGB")

        # Parse annotation
        target = self.parse_annotations(ann_path)

        # If no objects are found, return empty tensors
        if target['boxes'].shape[0] == 0:
            target['boxes'] = torch.empty((0, 4))
            target['labels'] = torch.empty(0, dtype=torch.int64)

        # Apply transforms
        if self.transforms:
            img = self.transforms(img)

        return img, target

    def __len__(self):
        return len(self.images)
    def parse_annotations(self, path):
            """Parses XML annotations into a target dictionary."""
            tree = ET.parse(path)
            root = tree.getroot()
            boxes = []
            labels = []

            for obj in root.findall('object'):
                label = 1  # Assuming a single class; adjust for multi-class datasets
                labels.append(label)
                bndbox = obj.find('bndbox')
                xmin = int(bndbox.find('xmin').text)
                ymin = int(bndbox.find('ymin').text)
                xmax = int(bndbox.find('xmax').text)
                ymax = int(bndbox.find('ymax').text)
                boxes.append([xmin, ymin, xmax, ymax])

            print(f"Boxes: {boxes}, Labels: {labels}")  # Debug print statement

            if len(boxes) == 0:
                print(f"No objects found in annotation file: {path}")  # Debug print for empty annotations

            target = {
                'boxes': torch.tensor(boxes, dtype=torch.float32),
                'labels': torch.tensor(labels, dtype=torch.int64)
            }
            return target

def get_transforms():
    """Return transformations for the dataset."""
    return T.Compose([
        T.ToTensor(),
        T.Resize((512, 512)),
        T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])

In [5]:

# ==========================
# 2. Model Setup
# ==========================
def get_faster_rcnn_model(backbone='resnet50', num_classes=11):  # 10 classes + background
    """Return a Faster-RCNN model with the specified backbone."""
    if backbone == 'resnet18':
        backbone_model = torchvision.models.resnet18(pretrained=True)
    elif backbone == 'resnet50':
        backbone_model = torchvision.models.resnet50(pretrained=True)
    elif backbone == 'resnet101':
        backbone_model = torchvision.models.resnet101(pretrained=True)
    else:
        raise ValueError("Invalid backbone choice!")

    # Use backbone without the classification head
    backbone_model = torch.nn.Sequential(*list(backbone_model.children())[:-2])
    backbone_model.out_channels = 2048

    # RPN and ROI pooling
    anchor_generator = AnchorGenerator(
        sizes=((32, 64, 128, 256, 512),),
        aspect_ratios=((0.5, 1.0, 2.0),)
    )
    roi_pooler = torchvision.ops.MultiScaleRoIAlign(
        featmap_names=['0'],
        output_size=7,
        sampling_ratio=2
    )

    # Define FasterRCNN model
    model = FasterRCNN(
        backbone_model,
        num_classes=num_classes,  # Number of classes (including background)
        rpn_anchor_generator=anchor_generator,
        box_roi_pool=roi_pooler
    )
    return model



In [6]:

# ==========================
# 3. Training and Validation
# ==========================
def train_one_epoch(model, optimizer, data_loader, device):
    model.train()
    epoch_loss = 0
    for images, targets in tqdm(data_loader):
        images = [img.to(device) for img in images]
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

        loss_dict = model(images, targets)
        losses = sum(loss for loss in loss_dict.values())
        optimizer.zero_grad()
        losses.backward()
        optimizer.step()
        epoch_loss += losses.item()

    return epoch_loss / len(data_loader)

def validate_one_epoch(model, data_loader, device):
    model.eval()
    metrics = []
    with torch.no_grad():
        for images, targets in tqdm(data_loader):
            images = [img.to(device) for img in images]
            outputs = model(images)
            # Compute validation metrics (Placeholder: Replace with actual evaluation code)
            metrics.append(0)  # Placeholder
    return np.mean(metrics)



In [10]:

# ==========================
# 4. Main Training Loop
# ==========================
def main():
    # Configurations
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    batch_size = 4
    num_epochs = 10
    backbone = 'resnet50'

    # Paths
    image_dir = '/content/drive/MyDrive/mini project/images'
    annotation_dir = '/content/drive/MyDrive/mini project/annotations'

    # Dataset and DataLoader
    dataset_train = UATDDataset(image_dir, annotation_dir, transforms=get_transforms())
    train_loader = DataLoader(dataset_train, batch_size=batch_size, shuffle=True, collate_fn=lambda x: tuple(zip(*x)))

    # Model, Optimizer
    model = get_faster_rcnn_model(backbone, num_classes=11).to(device)  # 10 classes + background
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    epoch_losses = []
    # Training Loop
    for epoch in range(num_epochs):
        print(f"Epoch {epoch + 1}/{num_epochs}")
        train_loss = train_one_epoch(model, optimizer, train_loader, device)
        print(f"Train Loss: {train_loss:.4f}")


if __name__ == "__main__":
    main()


Epoch 1/10


  0%|          | 0/13 [00:00<?, ?it/s]

Boxes: [[474, 752, 516, 788]], Labels: [1]
Boxes: [[344, 783, 442, 842], [448, 673, 506, 724], [496, 817, 592, 887]], Labels: [1, 1, 1]
Boxes: [[346, 538, 394, 578]], Labels: [1]
Boxes: [[240, 1394, 334, 1478]], Labels: [1]


  8%|▊         | 1/13 [00:01<00:17,  1.47s/it]

Boxes: [[706, 670, 766, 755]], Labels: [1]
Boxes: [[342, 765, 400, 820], [372, 956, 428, 1004], [440, 978, 542, 1054]], Labels: [1, 1, 1]
Boxes: [[326, 540, 370, 577]], Labels: [1]
Boxes: [[372, 957, 436, 1004], [338, 768, 412, 823]], Labels: [1, 1]


...

Boxes: [[8, 579, 58, 624]], Labels: [1]
Boxes: [[426, 1205, 492, 1269], [406, 1020, 466, 1082]], Labels: [1, 1]
Boxes: [[630, 672, 692, 751], [544, 1010, 640, 1084]], Labels: [1, 1]
Boxes: [[372, 1097, 424, 1156], [524, 1026, 652, 1099]], Labels: [1, 1]


 92%|█████████▏| 12/13 [00:11<00:00,  1.07it/s]

Boxes: [[342, 765, 400, 820], [372, 956, 428, 1004], [440, 978, 542, 1054]], Labels: [1, 1, 1]
Boxes: [[366, 1528, 420, 1599]], Labels: [1]


100%|██████████| 13/13 [00:11<00:00,  1.09it/s]

Train Loss: 7.8916


YOLO

In [ ]:
pip install ultralytics
